In [4]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt



In [5]:
def get_CNN_model():
    model = Sequential()
    
    model.add(Conv2D(32, (5, 5), input_shape=(96, 96, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(30))
    
    return model;

In [6]:
def compile_CNN_model(model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
def train_CNN_model(model, X_train, y_train, epochs=100):
    return model.fit(X_train, y_train, epochs=epochs, batch_size=200, verbose=1, validation_split=0.2)

def save_CNN_model(model, filename):
    model.save(filename + '.h5')
    
def load_CNN_model(filename):
    return load_model(filename + '.h5')


In [7]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

def load_data(test=False):
    FTRAIN = 'FaceRecognitionFiles/data/training.csv'
    FTEST = 'FaceRecognitionFiles/data/test.csv'
    
    fname = FTEST if test else FTRAIN
    
    df = read_csv(os.path.expanduser(fname)) # load dataframes
    
    # The Image column has pixel values separated by space
    # convert the values to numpy arrays
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    
    df = df.dropna() # drop all rows that have missing values in them
    
    X = np.vstack(df['Image'].values) / 255.
    X = X.astype(np.float32)
    X = X.reshape(-1, 96, 96, 1) # return each images as 96 x 96 x 1
    
    if not test: # only FTRAIN has target columns
        y = df[df.columns[:-1]].values
        y = (y - 48.) / 48. # scale target coordinates to [-1, 1] (Normalizing)
        X, y = shuffle(X, y, random_state=42) # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None
        
    return X, y
            

In [8]:
import cv2

X_train, y_train = load_data()

print(X_train.shape)


(2140, 96, 96, 1)


In [9]:
model = get_CNN_model()

compile_CNN_model(model, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
history = train_CNN_model(model, X_train, y_train, 100)

# train_my_CNN_model returns a History object. History.history attribute is a record of training loss values and metrics
# values at successive epochs, as well as validation loss values and validation metrics values (if applicable).

In [ ]:
save_CNN_model(model, 'FaceRecognitionTrainedModel')

In [1]:
import tensorflow as tf
import cv2
import numpy as np

def load_CNN_model(filename):
    return tf.keras.models.load_model(filename + '.h5')

In [2]:
model = load_CNN_model('FaceRecognitionTrainedModel')


In [3]:
cameraIndex = 0
camera = cv2.VideoCapture(cameraIndex)

cascadePath = 'FaceRecognitionFiles\cascades\haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(cascadePath)


while True:
    (gotImage, frame) = camera.read()
    frame2 = np.copy(frame)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.25, 6)
    
    #frame = cv2.rectangle(frame, (500,10), (620,65), (235,50,50), -1)
    
    for (x, y, w, h) in faces:
        
        gray_face = gray[y:y+h, x:x+w]
        color_face = frame[y:y+h, x:x+w]
        
    
        gray_normalized = gray_face / 255.

        original_shape = gray_face.shape

        face_resized = cv2.resize(gray_normalized, (96,96), interpolation=cv2.INTER_AREA)
        face_resized_copy = face_resized.copy()

        face_resized = face_resized.reshape(1, 96, 96, 1)

        keypoints = model.predict(face_resized)

        # De-Normalize the keypoints values
        keypoints = keypoints * 48 + 48

        face_resized_color = cv2.resize(color_face, (96,96), interpolation=cv2.INTER_AREA)
        face_resized_color2 = np.copy(face_resized_color)

        points = []

        for i, co in enumerate(keypoints[0][0::2]):
            points.append((co, keypoints[0][1::2][i]))

            
        frame[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)
        
        # Add KEYPOINTS to the frame2
        for keypoint in points:
            cv2.circle(face_resized_color2, keypoint, 1, (0,255,0), 1)     
            
        frame2[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, interpolation = cv2.INTER_CUBIC)

    cv2.imshow("Original Image", frame)
    cv2.imshow("keypoints", frame2)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        

camera.release()
cv2.destroyAllWindows()
    
    